# Decorators in Python

> "A intro summary of Python decorators"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Craig Stanton
- hide: false
- categories: [python, decorators]

Decorators are awesome - but how do they actually work? 

This is an overview of my takeaways from a deep dive into decorators. It does not cover all concepts of decorators, nor does it reveal anything revolutionary. My goal with this post is to summarize the concepts that will help to build and troubleshoot decorators in our code, knowing full well that we will probably need to refer to the specific formatting of decorator design when writing in a project.

### TL;DR

* Decorators are functions that take functiions as arguments in order to change the behaviour of the passed function
* You write decorators **quite differently** if the decorator accepts arguments vs if it does not accept any arguments
    * A decorator with no arguments: has 1 inner function (commonly named `wrapper` but there are not rules regarding this that I've seen), and it returns the wrapper **object** (*not the called function*)
        ```python
        def decorator(func):
            def wrapper(*args, **kwargs):
                print("Can do something here before the passed function is called")
                result = func(*args, **kwargs)
                print("You can modify the result of the function output here")
                return result
            return wrapper
        ```
    * A decorator that accepts arguments has **2 inner functions**:
        ```python
        def decorator_args(bool_arg: bool = False) -> str:
            def inner(func):                      # inner accepts func
                def wrapper(*args, **kwargs):     # wrapper accepts args and kwargs for func
                    if bool_arg:
                        print("Conditionally update a function based on the decorator argument")
                        return func(*args, **kwargs).upper()
                    return func(*args, **kwargs).lower()
                return wrapper
            return inner
        ```
* Once you understand function decorators, class decorators are *sort of* a logical extension using class magic methods instead
    * A class decorator with no arguments:
        ```python
        class decorator:
            def __init__(self, func):
                self.func = func                    # func is processed in __init__

            def __call__(self, *args, **kwargs):    # __call__ acts as an inner wrapper function; accepts args for func
                def wrapper(*args, **kwargs):
                    print("Can do something here")
                    return func(*args, **kwargs)
                return wrapper
        ```
    * A class decorator with args:
        ```python
        class decorator_args:
            def __init__(self, *decorator_args, **decorator_kwargs):    # func is not passed in init; it is passed in __call__
                self.arg1 = decorator_args[0]
                self.arg2 = decorator_kwargs["arg2"]

            def __call__(self, func):
                def wrapper(*args, **kwargs):
                    print("Can modify the func based on decorator args")    
                    return func(*args, **kwargs)
                return wrapper
        ```
    * Important difference with class decorators:
        * Despite its name, `__call__` is only called once at instantiation. The inner `wrapper` function is what is called every subsequent call.
* A decorator typically returns a function *object* (on the callable function)


### Example Setup

Let's create a decorator that modifies the output of an API response. This is based on a real use case I am working on, where the code calls different endpoints of the *same API service*, meaning the responses and errors are standardized across the various endpoints.

Instead of building in parsing and error handling into each call, I use a decorator to handle this one time.

First let's look at a sample response:

In [5]:
import requests

In [6]:
base_url = "https://randomuser.me/api/"

def api_single_user():
    response = requests.get(base_url)
    return response.json()

api_single_user()

{'results': [{'gender': 'female',
   'name': {'title': 'Miss', 'first': 'محیا', 'last': 'حسینی'},
   'location': {'street': {'number': 2649, 'name': 'میدان قیام'},
    'city': 'خوی',
    'state': 'مازندران',
    'country': 'Iran',
    'postcode': 64139,
    'coordinates': {'latitude': '29.7035', 'longitude': '148.9815'},
    'timezone': {'offset': '+10:00',
     'description': 'Eastern Australia, Guam, Vladivostok'}},
   'email': 'mhy.hsyny@example.com',
   'login': {'uuid': '913f4c6a-9f76-48dd-bcd8-02d4952f94c4',
    'username': 'orangebird224',
    'password': 'sinclair',
    'salt': '9EAHZ37U',
    'md5': '8b84bf452f8acd65c1810dc636425b37',
    'sha1': 'c09d34e11d135ef304e45afaadbf2e9648a5e416',
    'sha256': '3100f6d098cd5e95dd6a633399873a96bff426c0a0496f56925456eab6fe0619'},
   'dob': {'date': '1991-10-20T18:10:11.390Z', 'age': 31},
   'registered': {'date': '2007-03-07T14:25:53.978Z', 'age': 15},
   'phone': '031-96272721',
   'cell': '0906-402-0122',
   'id': {'name': '', 'value

The response is a dictionary. In practice, my experience is that we should be using a `dataclass` or a Pydantic model to manage dictionary responses. However we will keep as a dict for this exercise.

### Function decorators

If we know that we will always need to call either the `.lower()` or `upper()` string method following every API call, we could use a decorator to do this for us. 

Why even do this simple code with a decorator?

Aside from apeasing the DRY Python methodology, I find decorators are a nice visual queue when reading code - seeing a decorator above a function is easier to read than burried code within each function, in my opinion.

##### Creating Function Decorators

So how do we create different types of decorators?

Below we write a decorator that 1) doesn't accept arguments and 2) does accept arguments

In [7]:
def lower_no_args(func) -> str:
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs).lower()
    return wrapper

In [8]:

def lower_args(caps: bool = False) -> str:
    def inner(func):                       # inner accepts func
        def wrapper(*args, **kwargs):      # wrapper accepts args and kwargs for func
            if caps:
                return func(*args, **kwargs).upper()
            return func(*args, **kwargs).lower()
        return wrapper
    return inner

Let's see how each of these work

In [9]:
@lower_no_args
def single_user_firstname():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname()

'norberta'

In [10]:
@lower_args()         # notice the callable
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()


'terry'

In [11]:

@lower_args(caps=True)
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()


'DICKY'

#### To Summarize

You will have seen that the `lower_args` decorator accepts an argument. As a result - there is a **second `inner` function** that *accepts the passed function as an argument*.

This was important for me to realize - we are now dealing with 3 functions *when the decorator accepts arguments*: the decorator itself, the inner function that accepts the function to decorate, and finally the wrapper function that accepts the arguments passed to the decorated function. This is a lot to...wrap... your head around. However there is no sense in memorizing this since you can look the details up any time - what is helpful to remember is simply:

> If a decorator accepts arguments (required or optional), there is a *second internal function* that you must build.

As mentioned in [this awesome post](https://aikikode.me/blog/python-decorators-manual/#asynchronous-decorators), you can also make decorators that optionally accept arguments. 

### Class Decorators

Let's do the same thing but with class decorators

In [12]:
class lower_no_args:
    def __init__(self, func):
        self.func = func            # func is processed in __init__

    def __call__(self, *args, **kwargs):   # __call__ acts as an inner wrapper function
        return self.func(*args, **kwargs).lower()

In [13]:
class lower_args:
    def __init__(self, caps: bool = False):
        self.caps = caps            # func is not passed in init; it is passed in __call__

    def __call__(self, func):
        def wrapper(*args, **kwargs):   # wrapper accepts args and kwargs for func
            if self.caps:
                return func(*args, **kwargs).upper()
            return func(*args, **kwargs).lower()
        return wrapper

One thing to notice is that naming convention for class decorators does not follow normal "title case" for classes. 

Let's again see how each of these work

In [15]:
@lower_no_args
def single_user_firstname():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname()

__call__ called with args: () and kwargs: {}


'philip'

In [16]:
@lower_args()
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()

'giray'

In [17]:
@lower_args(caps=True)
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()

'ROLAND'

As you can see, if the decorator needs to receive arguments, there is a second inner function we must define. The only difference is that the first inner function for a class decorator is the `__call__` method.

OK so why would you ever use a class decorator if it behaves the same as a function decorator?

There are two scenarios I have seen where class decorators are useful:
1. If the decorator needs to track soome sort of state
2. Using classmethods to derive specific behaviour while using a standard base class (ie. `@modify.upper()` and `@modify.lower()` would be classmethods of the base `modify` class decorator)

#### Class Decorator Usage Examples

Lets add some sort of state that we want our decorator to keep track of. Let's imagine counting the number of times the API was called - this is a basic example but could be used as a mechanism to throttle the average API calls per minute.

In [82]:
class lower_args:
    def __init__(self, caps: bool = False):
        self.caps = caps            # func is not passed in init; it is passed in __call__
        self.num_calls = 0

    def __call__(self, func):
        def wrapper(*args, **kwargs):   # wrapper accepts args and kwargs for func
            if self.caps:
                return func(*args, **kwargs).upper()
            return func(*args, **kwargs).lower()
        self.num_calls += 1
        print(f"Num calls: {self.num_calls}")
        return wrapper

In [80]:
@lower_args(caps=True)
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()

1282.0


'NEUSA'

In [81]:
single_user_firstname_caps()
single_user_firstname_caps()

'MATTHEW'

Why didn't the second and third API calls print the statement?

It turns out that just like `__init__`, the `__call__` method is only called **once**. As a decorator, this means only the first function call triggers these methods. 

> Once the class generator has been instantiated (ie. used once), only the inner `wrapper` function is called subsequently.

As a result, any update to the class state must be embedded in the `wrapper`:

In [21]:
class lower_args:
    def __init__(self, caps: bool = False):
        self.caps = caps            # func is not passed in init; it is passed in __call__
        self.num_calls = 0

    def __call__(self, func):
        def wrapper(*args, **kwargs):   # wrapper accepts args and kwargs for func
            self.num_calls += 1
            print(f"Number of calls made: {self.num_calls}")
            if self.caps:
                return func(*args, **kwargs).upper()
            return func(*args, **kwargs).lower()
        
        return wrapper

In [61]:
@lower_args(caps=True)
def single_user_firstname_caps():
    return api_single_user()["results"][0]["name"]["first"]

single_user_firstname_caps()

Number of calls made: 1


'CHARLES'

In [62]:
single_user_firstname_caps()
single_user_firstname_caps()

Number of calls made: 2
Number of calls made: 3


'KATIE'

We now have a decorator that can keep track of the number of times it has been called across our application. If we were to combine this with a `datetime` state, we could build a while loop to pause the function if we have hit too high of an API call rate.

Of course there is plenty more to look at with respect to class decorators (and decorators in general), including asyncronous decorators

## Summary

For the longest time I avoided learning about the *inner* workings of decorators. And while I fully anticipate having to come back to these examples whenever I create a decorator, the easy to remember takeaways that will help me design and troubleshoot code are:

1. Both function and class decorators require an additional inner function if the decorator is to take arguments
2. Decorators return the inner function *object* and not the function callable.
3. Class decorators have a required inner function called `__call__` and it is only called once (upon instantiation) - the fuction returned within `__call__` is what is triggered in each subsequent call.

## References

Heavily relied upon [Denis Kovalev's post](https://aikikode.me/blog/python-decorators-manual/#asynchronous-decorators) to make sense of all of this. This blog was for my own learning to boil down the concepts I took away from their very insightful blog.